# 2. Data Preprocessing and Cleaning

This notebook takes the raw Lending Club dataset and performs a series of preprocessing and cleaning steps. The goal is to create a clean, model-ready dataset by:

1.  Filtering for relevant loan outcomes.
2.  Creating a binary target variable for default prediction.
3.  Removing features that would cause data leakage.
4.  Handling missing values.
5.  Cleaning and transforming data types.
6.  Engineering a new feature for credit history length.
7.  Dropping unnecessary identifier and text columns.

The final cleaned DataFrame will be saved to a new CSV file.

In [ ]:
import pandas as pd
import re
import os

## Configuration

Define the input file (the raw dataset) and the output file for our cleaned data.

In [ ]:
# --- Configuration ---
INPUT_FILE = 'lc_accepted_loans_full_2007to2018.csv'
OUTPUT_FILE = 'lc_loans_cleaned.csv'

## Step 1: Load Raw Dataset

Load the raw data by uploading the `lc_accepted_loans_full_2007to2018.csv` file.

In [ ]:
try:
    df = pd.read_csv(INPUT_FILE, low_memory=False)
    print("1. Loading raw dataset...")
    print(f"   - Initial shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: The file '{INPUT_FILE}' was not found.")
    print("Please make sure you've uploaded the file to this Colab session.")

## Step 2: Filter and Encode the Target Variable

For our predictive model, we only need loans that have reached a final outcome. We will filter the dataset to keep only loans that are **'Fully Paid'** (good loans) or **'Charged Off'** (bad loans).

We will then create a new binary `target` column where `1` represents a 'Charged Off' loan and `0` represents a 'Fully Paid' loan.

In [ ]:
print("\n2. Handling Target Variable ('loan_status')...")

# Keep loans with a terminal status only ("Charged Off", "Fully Paid")
terminal_statuses = ['Fully Paid', 'Charged Off']
df = df[df['loan_status'].isin(terminal_statuses)].copy()
print(f"   - Shape after keeping terminal statuses: {df.shape}")

# Create a binary target variable
df['target'] = df['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)
df = df.drop(columns=['loan_status'])
print("   - Created binary 'target' column and dropped original 'loan_status'.")

## Step 3: Remove Columns with Data Leakage

To build a realistic model, we must remove any features that contain information that would not have been available at the time of a loan application. These are often called "performance variables" and their inclusion would cause data leakage, leading to an overly optimistic model.

In [ ]:
print("\n3. Removing columns that cause data leakage (performance variables)...")
# These columns contain information about borrower behavior after the loan was funded.
leaky_cols = [
    'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
    'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
    'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
    'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
    'last_fico_range_high', 'last_fico_range_low'
]
df = df.drop(columns=leaky_cols, errors='ignore')
print(f"   - Dropped {len(leaky_cols)} leaky columns.")
print(f"   - Shape after dropping leaky columns: {df.shape}")

## Step 4: Handle Missing Values

We will handle missing data using a two-step strategy:
1.  **Drop Sparse Columns:** First, we remove any column that is missing more than 30% of its values.
2.  **Drop Rows with NaNs:** After removing the sparsest columns, we drop any remaining rows that still contain missing values.

In [ ]:
print("\n4. Handling missing values...")

# Drop columns with more than 30% missing data
missing_threshold = 0.30
missing_fractions = df.isnull().mean()
cols_to_drop_by_missing = missing_fractions[missing_fractions > missing_threshold].index
df = df.drop(columns=cols_to_drop_by_missing)
print(f"   - Dropped {len(cols_to_drop_by_missing)} columns with >{missing_threshold*100}% missing values.")
print(f"   - Shape after dropping sparse columns: {df.shape}")

# Drop any remaining rows with missing data
rows_before_drop = len(df)
df = df.dropna().copy()
rows_after_drop = len(df)
print(f"   - Dropped {rows_before_drop - rows_after_drop} rows with remaining missing values.")
print(f"   - Shape after dropping rows with NaNs: {df.shape}")

## Step 5: Clean Data Types and Feature Engineering

We keep some key features to make them usable in machine learning. This includes converting text-based numbers into integers and engineering a new feature for the length of the borrower's credit history.

In [ ]:
print("\n5. Cleaning data types and engineering features...")

# Clean 'term' column ('36 months' -> 36)
df['term'] = df['term'].str.extract(r'(\d+)').astype(int)

# Clean 'emp_length' column ('10+ years' -> 10, '< 1 year' -> 0)
def clean_emp_length(length_str):
    if pd.isna(length_str): return None
    if length_str == '10+ years': return 10
    if length_str == '< 1 year': return 0
    match = re.search(r'\d+', str(length_str))
    if match:
        return int(match.group())
    return None

df['emp_length'] = df['emp_length'].apply(clean_emp_length)

# errors='coerce' will force any bad dates into NaT (Not a Time)
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], errors='coerce')
df['issue_d'] = pd.to_datetime(df['issue_d'], errors='coerce')

# Drop any rows that failed the date conversion (now have NaT)
# or failed the emp_length conversion
rows_before = len(df)
df.dropna(subset=['earliest_cr_line', 'issue_d', 'emp_length'], inplace=True)
rows_after = len(df)
print(f"   - Dropped {rows_before - rows_after} rows due to bad date or emp_length values.")

# Covert the columns into datetime objects
# to calculate the credit history length feature
df['credit_history_length_days'] = (df['issue_d'] - df['earliest_cr_line']).dt.days

print("   - Cleaned 'term' and 'emp_length' columns.")
print("   - Created 'credit_history_length_days' feature.")

## Step 6: Drop Other Unnecessary Columns

Finally, we remove columns that are not useful for a predictive model, such as unique identifiers, URLs, and free-text fields.

In [ ]:
print("\n6. Dropping other unnecessary columns (IDs, free text, etc.)...")
unnecessary_cols = [
    'id', 'member_id', 'url', 'title', 'emp_title', 'zip_code',
    'addr_state', 'issue_d', 'earliest_cr_line' # Drop original date columns
]
df = df.drop(columns=unnecessary_cols, errors='ignore')
print(f"   - Dropped {len(unnecessary_cols)} unnecessary columns.")

## Final Step: Save the Cleaned Dataset

The preprocessing is complete. We will now save the cleaned and transformed DataFrame to a new CSV file. We'll also display a summary of the final dataset to confirm the data types and column count.

In [ ]:
print(f"\n7. Saving cleaned data...")

# Save the final cleaned DataFrame
df.to_csv(OUTPUT_FILE, index=False)

print(f"   - Final shape of cleaned data: {df.shape}")
print(f"   - Success! Cleaned dataset saved to '{OUTPUT_FILE}'")
print("\n--- Data Preprocessing Complete ---")

# Display a summary of the final DataFrame
print("\nFinal DataFrame Info:")
df.info()